# Julia – a modern approach to scientific computing

![](julia-logo-325-by-225.png)

https://www.julialang.org



Evaluate the following cell by selecting it and then hit Shift+Return. Watch the top right status indicator of the Jupyter Notebook kernel to have started up Julia. The result will appear below.

In [1]:
VERSION

v"1.0.3"

Vectors and matrices have a simple syntax. Notice, that the `'` means to transpose this vector.

In [4]:
import Pkg
Pkg.build("Ipopt")
using Ipopt

  Building 

Ipopt → `~/.julia/packages/Ipopt/OLtKb/deps/build.log`


┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1192


In [4]:
A = [  1   9   1
       0   1   0
    -1.1   0  -1]

3×3 Array{Float64,2}:
  1.0  9.0   1.0
  0.0  1.0   0.0
 -1.1  0.0  -1.0

… and here we solve this as a linear system of equations: $A \cdot x = b$

In [4]:
x = A \ b

3×1 Array{Float64,2}:
  476.0
    4.4
 -514.6

In [5]:
A * x

3×1 Array{Float64,2}:
  1.0
  4.4
 -9.0

## Batman Curve

Transcribed from _Julia: A Fast Language for Numerical Computing_,
by Alan Edelman,  
[SIAM News, Volume 49 | Number 02 | March 2016](https://sinews.siam.org/Details-Page/julia-a-fast-language-for-numerical-computing-1)

![](batman-demo-300.png)

In [0]:
# Note: the first time in a project, it can take over a minute to precompile PyPlot
using PyPlot
function batman()

    # bat functions: semicircle, ellipse, shoulders, bottom, cowl
    # harmless Compat.UTF8String warnings from julia kernel on first run
    σ(x) = sqrt.(1-x.^2)
    e(x) = 3σ(x/7)
    s(x) = 4.2 - .5x - 2.8σ(.5x-.5)
    b(x) = σ(abs.(2-x)-1)-x.^2/11 + .5x - 3
    c(x) = [1.7, 1.7, 2.6, .9]

    # plot symmetrically across y-axis
    p(x,f) = plot(-x,f(x), color="black") , plot(x,f(x), color="black")
    p((3:.1:7),e);p(4:.1:7,t->-e(t))   # ellipse
    p(1:.1:3,s);p(0:.1:4,b)            # shoulders and bottom
    p([0,.5,.8,1],c)
end
batman();

In [7]:
using JuMP
using Ipopt

mod = Model(solver = IpoptSolver())

# Parameters
vol_min = 1.0/8
sol_max = 10.0
num_iter = 10

@variable(mod, b[1:3] >= 0.0)

# Objective function
@NLexpression(mod, series, 1/((b[1] - b[2])*b[1]*(b[2] + b[3] - 4)*b[3]) - 1/((b[1] - b[2])*(b[1] + b[3] - 4)*b[2]*b[3]))
# series = 1/((b[1][1] - b[2][1])*b[1][1]*(b[2][1] + b[3][1] - 4)*b[3][1]) - 1/((b[1][1] - b[2][1])*(b[1][1] + b[3][1] - 4)*b[2][1]*b[3][1])

@NLconstraint(mod, b[1:3][1] <= sol_max)
@NLconstraint(mod, series >= vol_min)
@NLconstraint(mod, series <= 1.0)
@NLobjective(mod, Min, series)

mod

In [8]:
# Solve for the control and state
println("Solving...")
status = solve(mod)

# Display results
println("Solver status: ", status)
println("Max height: ", getObjectiveValue(mod))

Solving...



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        7
Number of nonzeros in Lagrangian Hessian.............:       18

The inequality constraints contain an invalid number

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality 

┌ Warning: Ipopt finished with status Invalid_Number_Detected
└ @ Ipopt /home/user/.julia/packages/Ipopt/OLtKb/src/MPB_wrapper.jl:178
┌ Warning: Not solved to optimality, status: Error
└ @ JuMP /ext/julia/julia/depot/packages/JuMP/PbnIJ/src/nlp.jl:1283


Max height: 0

┌ Warning: `getObjectiveValue` is deprecated, use `getobjectivevalue` instead.
│   caller = top-level scope at In[8]:7
└ @ Core In[8]:7


.0
